In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')

sns.set_style("whitegrid")
from torch.utils.data import random_split

import pandas as pd




In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform  
)


torch.manual_seed(42)
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size


train_ds, valid_ds = random_split(dataset, [train_size, valid_size])
len(train_ds), len(valid_ds)
    

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(batch)

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()


    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

In [ ]:
from torchvision import models
class Resnet18_1(nn.Module):
    def __init__(self):
        super().__init__()
        resnet18_pre = models.resnet18(pretrained=True)
        self.conv_layers = nn.ModuleList()
        for child in resnet18_pre.named_children():
            if child[0] == "fc":
                break
            self.conv_layers.append(child[1])

        self.fc1 = nn.Linear(resnet18_pre.fc.in_features, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):

        for layer in self.conv_layers:
            x = layer(x)

        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

## Resnet18 with all frozen layers. Classifier: flatten -> fc(120 params) -> fc

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:

batch_size = 8
epochs = 30
learning_rate = 1e-3

data = np.array([[0]*4]*epochs)
resnet18_1 = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])

train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

#Freeze all layers
net = Resnet18_1().to(device)
for param in net.conv_layers.parameters():    
    param.requires_grad = False


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")

    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)
    resnet18_1.loc[t,"test_acc"] = test_acc
    resnet18_1.loc[t,"train_acc"] = train_acc
    resnet18_1.loc[t,"test_loss"] = test_loss
    resnet18_1.loc[t,"train_loss"] = train_loss

# resnet18_1.to_csv("../data/results/resnet18_1/ADAM_bs8_epoch30_lr-3/resnet18_1.csv")

resnet18_1.to_csv("adam_bs8_epoch30_lr-3.csv")
!cp adam_bs8_epoch30_lr-3.csv "drive/My Drive/DL/Resnet18_1/"
 
    

In [ ]:
torch.save(net.state_dict(), '../models/resnet18_1.pth')